# Data Mining Project3——PageRank
### （1）读取数据

In [1]:
import numpy as np
import pandas as pd
import random

def loadData():
    Data = pd.read_csv("FBtree_TestingData.csv")       #使用pandas读取数据
    newData=np.zeros((len(Data,),2),dtype=int)
    for i in range(len(Data)):                       
        newData[i]=Data['items'][i].split(" ")           #对数据进行切割
    #print(newData)
    return newData


data1=pd.read_csv("graph_1.txt",index_col=["I"])
data2=pd.read_csv("graph_2.txt",index_col=["I"])
data3=pd.read_csv("graph_3.txt",index_col=["I"])
data4=pd.read_csv("graph_4.txt",index_col=["I"])
data5=pd.read_csv("graph_5.txt",index_col=["I"])
data6=pd.read_csv("graph_6.txt",index_col=["I"])
#单向数据
load_data=loadData()
ind =load_data[:,0]
data7=pd.DataFrame(load_data[:,1],index=ind,columns=['O'])
#print(data7)
#双向数据
ind =np.append(load_data[:,0],load_data[:,1])
bi_data=np.append(load_data[:,1],load_data[:,0])
data8=pd.DataFrame(bi_data,index=ind,columns=['O'])
#print(data8)

In [2]:
#更改后的数据
data1_1=pd.read_csv("graph_1-1.txt",index_col=["I"])
data1_2=pd.read_csv("graph_1-2.txt",index_col=["I"])
data2_1=pd.read_csv("graph_2-1.txt",index_col=["I"])
data3_1=pd.read_csv("graph_3-1.txt",index_col=["I"])

## （2）数据预处理
1、获取最大网页编号

2、把读取的数据转化为字典型

3、根据字典数据转化成路径矩阵data_matrix


In [3]:
def size(data):
    index_max=data.index.max()
    data_max=data.O.max()
    if index_max>data_max:
        max_number=index_max
    else:
        max_number=data_max
    return max_number
    
def data2dic(data):
    index=data.index.tolist()
    index_tostr=[str(i) for i in index]
    datas=data["O"]
    data_tostr=[str(i) for i in datas]
    dits={}
    j=0
    for i in index_tostr:
        dits.setdefault(i,[]).append(data_tostr[j])
        j=j+1
    return dits

def dic2mat(dic,size):
    data_matrix=np.zeros((size,size))
    for key,values in dic.items():
        for value in values:
            key=int(key)
            value=int(value)
            data_matrix[key-1][value-1]=1
            #print(key,value)
            #print(data_matrix)
    return data_matrix

In [4]:
data_matrix1=dic2mat(data2dic(data1),size(data1))
data_matrix2=dic2mat(data2dic(data2),size(data2))
data_matrix3=dic2mat(data2dic(data3),size(data3))
data_matrix4=dic2mat(data2dic(data4),size(data4))
data_matrix5=dic2mat(data2dic(data5),size(data5))
data_matrix6=dic2mat(data2dic(data6),size(data6))
data_matrix7=dic2mat(data2dic(data7),size(data7))

In [5]:
data_matrix1_1=dic2mat(data2dic(data1_1),size(data1_1))
data_matrix1_2=dic2mat(data2dic(data1_2),size(data1_2))
data_matrix2_1=dic2mat(data2dic(data2_1),size(data2_1))
data_matrix3_1=dic2mat(data2dic(data3_1),size(data3_1))

## PageRank 实现过程

In [6]:
#N = 8    #八个网页
d = 0.85 #阻尼因子为0.85
delt = 0.00001#迭代控制变量
#两个矩阵相乘
def matrix_multi(A,B):
    result = [[0]*len(B[0]) for i in range(len(A))]
    for i in range(len(A)):
        for j in range(len(B[0])):
            for k in range(len(B)):
                result[i][j] += A[i][k]*B[k][j] 
    return result
 
#矩阵A的每个元素都乘以n
def matrix_multiN(n,A):
    result = [[1]*len(A[0]) for i in range(len(A))]
    for i in range(len(A)):
        for j in range(len(A[0])):
            result[i][j] = n*A[i][j]
    return result
 
#两个矩阵相加
def matrix_add(A,B):
    if len(A[0])!=len(B[0]) and len(A)!=len(B):
        return
    result = [[0]*len(A[0]) for i in range(len(A))] 
    for i in range(len(A)):
        for j in range(len(A[0])):
            result[i][j] = A[i][j]+B[i][j]
    return result 
 
def pageRank(A,N):
    e = []
    for i in range(N):
        e.append(1) 
    norm = 100
    New_P = []
    for i in range(N):
        New_P.append([random.random()]) 
    r = [ [(1-d)*i*1/N] for i in e]
    while norm > delt:
        P = New_P
        New_P = matrix_add(r,matrix_multiN(d,matrix_multi(A,P))) #P=(1-d)*e/n+d*M'P PageRank算法的核心
        norm = 0
        #求解矩阵一阶范数
        for i in range(N):
            norm += abs(New_P[i][0]-P[i][0])
    for i in range(len(New_P)):
        print("第",i+1,"个网页的pagepank值为：",New_P[i])
    #print(New_P)

#根据邻接矩阵求转移概率矩阵并转向
def tran_and_convert(A):
    result = [[0]*len(A[0]) for i in range(len(A))]
    result_convert = [[0]*len(A[0]) for i in range(len(A))]
    for i in range(len(A)):
        for j in range(len(A[0])):
            if sum(A[i])!=0:
                result[i][j] = A[i][j]*1.0/sum(A[i])
            else:
                result[i][j]=0
    for i in range(len(result)):
        for j in range(len(result[0])):
            result_convert[i][j]=result[j][i]
    return result_convert

In [7]:
def display(data, title):
    print(title)
    print("----------------------------------我是分割线------------------------------------")
    Matrix=tran_and_convert(data.tolist())
    pageRank(Matrix,len(data))
    print("\n")

In [8]:
display(data_matrix1,"数据集1")
display(data_matrix2,"数据集2")
display(data_matrix3,"数据集3")
display(data_matrix4,"数据集4")
display(data_matrix5,"数据集5")
display(data_matrix6,"数据集6")
display(data_matrix7,"数据集:project1单向")
display(data_matrix8,"数据集:project1双向")

数据集1
----------------------------------我是分割线------------------------------------
第 1 个网页的pagepank值为： [0.025000000000000005]
第 2 个网页的pagepank值为： [0.04625000000000001]
第 3 个网页的pagepank值为： [0.06431250000000001]
第 4 个网页的pagepank值为： [0.079665625]
第 5 个网页的pagepank值为： [0.09271578125]
第 6 个网页的pagepank值为： [0.1038084140625]


数据集2
----------------------------------我是分割线------------------------------------
第 1 个网页的pagepank值为： [0.20000615638131414]
第 2 个网页的pagepank值为： [0.2000040549709436]
第 3 个网页的pagepank值为： [0.2000020473425035]
第 4 个网页的pagepank值为： [0.20000380356429487]
第 5 个网页的pagepank值为： [0.20000482280103635]


数据集3
----------------------------------我是分割线------------------------------------
第 1 个网页的pagepank值为： [0.17543858131888013]
第 2 个网页的pagepank值为： [0.3245642896604555]
第 3 个网页的pagepank值为： [0.3245613731640761]
第 4 个网页的pagepank值为： [0.1754400395670698]


数据集4
----------------------------------我是分割线------------------------------------
第 1 个网页的pagepank值为： [0.2803031196178134]
第 2 个网页的pagepank值为： [

第 337 个网页的pagepank值为： [0.00034809232825127283]
第 338 个网页的pagepank值为： [0.0003393394210394394]
第 339 个网页的pagepank值为： [0.00033150604578486317]
第 340 个网页的pagepank值为： [0.00033150604578486317]
第 341 个网页的pagepank值为： [0.0003677031731418828]
第 342 个网页的pagepank值为： [0.0003393394210394394]
第 343 个网页的pagepank值为： [0.0003979139754821653]
第 344 个网页的pagepank值为： [0.000407232190429929]
第 345 个网页的pagepank值为： [0.0003677031731418828]
第 346 个网页的pagepank值为： [0.00034557600529717486]
第 347 个网页的pagepank值为： [0.0003585222405193421]
第 348 个网页的pagepank值为： [0.001373806378999167]
第 349 个网页的pagepank值为： [0.00036793210655938156]
第 350 个网页的pagepank值为： [0.0005143347090369072]
第 351 个网页的pagepank值为： [0.000407232190429929]
第 352 个网页的pagepank值为： [0.00033150604578486317]
第 353 个网页的pagepank值为： [0.0003665680286851849]
第 354 个网页的pagepank值为： [0.00034194075393709314]
第 355 个网页的pagepank值为： [0.0003405587958352146]
第 356 个网页的pagepank值为： [0.00033277671337191597]
第 357 个网页的pagepank值为： [0.00033150604578486317]
第 358 个网页的pagepank值为： [0.000

KeyboardInterrupt: 

In [9]:
display(data_matrix1_1,"数据集1-1")
display(data_matrix1_2,"数据集1-2")
display(data_matrix2_1,"数据集2-1")
display(data_matrix3_1,"数据集3-1")

数据集1-1
----------------------------------我是分割线------------------------------------
第 1 个网页的pagepank值为： [0.025000000000000005]
第 2 个网页的pagepank值为： [0.035625000000000004]
第 3 个网页的pagepank值为： [0.06590625000000001]
第 4 个网页的pagepank值为： [0.08102031250000001]
第 5 个网页的pagepank值为： [0.09386726562500002]
第 6 个网页的pagepank值为： [0.10478717578125002]


数据集1-2
----------------------------------我是分割线------------------------------------
第 1 个网页的pagepank值为： [0.025000000000000005]
第 2 个网页的pagepank值为： [0.029250000000000005]
第 3 个网页的pagepank值为： [0.05411250000000001]
第 4 个网页的pagepank值为： [0.07524562500000001]
第 5 个网页的pagepank值为： [0.09320878125000003]
第 6 个网页的pagepank值为： [0.10847746406250003]


数据集2-1
----------------------------------我是分割线------------------------------------
第 1 个网页的pagepank值为： [0.2151522445155027]
第 2 个网页的pagepank值为： [0.12144046241449613]
第 3 个网页的pagepank值为： [0.22466570707438044]
第 4 个网页的pagepank值为： [0.22096763683366397]
第 5 个网页的pagepank值为： [0.21782428640721305]


数据集3-1
---------------------